# Ingest

This workbook will ingest an excel file (name is configured in config.json), does some basic clean-up and stores a csv dataframe with function save_df_to_csv that stores the files with folder structure "base_folder / YYYY / MM / DD / HH / MM / SS"

TODO: Add example config file

```json
{
    "name_mapping": {
        "Fecha": "date",
        "Valor": "value",
        "Observaciones": "observations",
        "Importe": "amount",
        "Divisa": "amount_currency",
        "Saldo": "balance",
        "Divisa.1": "balance_currency",
        "Nº mov": "movement_number",
        "Oficina": "office",
        "Categoria": "category",
        "Subcategoria": "subcategory",
        "Detalle": "detail",
        "Nº Factura": "invoice_number",
        "Referencia archivo factura": "invoice_file_reference",
        "DATOS": "data"
    },
    "columns_to_keep": ["date", "observations", "amount", "balance", "category", "subcategory", "detail", "invoice_number", "invoice_file_reference", "data"],
    "local_path": "your file path",
    "file_name": "your file name",
    "sheet_name": "Excel file sheet name",
    "ingestion_output_folder": "../data/raw",
    "ingestion_output_file": "../data/bronze"
}
```

The following packages are required:
* pip install openpyxl

In [12]:
# Import libraries
import json
import sys
sys.path.append("..")
from utils.filesystem import *

In [13]:
# Config file path
config_file = '../config.json'

# Load the config file
with open(config_file) as f:
    config = json.load(f)

# Load the basic config parameters
local_path = config['local_path']
file_name = config['file_name']
sheet_name = config['sheet_name']

# Load the config parameters (ingestion transformation)
name_mapping = config['name_mapping']
columns_to_keep = config['columns_to_keep']

# Load the ingestion folder locations
ingestion_output_folder = config['ingestion_output_folder']
ingestion_output_file = config['ingestion_output_file']

In [14]:
# Ignore UserWarning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load a specific worksheet from the Excel file
try:
    dataframe = load_excel_sheet_to_dataframe (local_path, file_name, sheet_name, 4)

    # Count the number of rows and columns
    rows, columns = dataframe.shape
    print(f"The dataframe has {rows} rows and {columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")

The dataframe has 2503 rows and 17 columns


In [15]:
# Rename the columns
dataframe = dataframe.rename(columns=name_mapping)

# Calculate the columns to drop as the difference between all columns and the columns to keep
columns_to_drop = dataframe.columns.difference(columns_to_keep)

# Drop the columns
dataframe = dataframe.drop(columns=columns_to_drop)

# Display the names of the columns
print(f"Columns: {dataframe.columns}")


Columns: Index(['date', 'observations', 'amount', 'balance', 'category', 'subcategory',
       'detail', 'invoice_number', 'invoice_file_reference'],
      dtype='object')


In [16]:
# Persist the dataframe to a CSV file
file_path = create_foldername(ingestion_output_folder, file_name.split('.')[0])
try:
    save_df_to_parquet(dataframe, file_path, "transactions")
    print(f"Dataframe saved to {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Dataframe saved to ../data/raw\EXTRACTO BANCO 2024\2024/05/19/23/53/19/output.parquet
